In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.graph_objs as go

In [2]:
country_codes = pd.read_csv('../countries_codes_and_coordinates.csv' , sep = ',', index_col = 0)

In [3]:
data = pd.read_csv('results/optimization_results_world_grid.csv')
data = data.set_index('Country')
data = data.drop(['Antarctica'], axis =0)
for col in data.columns[1:]: 
    data[f'{col}_normalized']=data[col]/data['mean_load']
    
for c in data.index.to_list(): 
    results = pd.read_pickle(f'results/{c}/optimization_results.pickle')
    iso_code = country_codes.loc[c,'Alpha-3 code' ].split(' ')[1]
    data.loc[c, 'capacity_factor_wind'] = results['E_wind']/data.loc[c, 'P_wind']/8760
    data.loc[c, 'capacity_factor_pv'] = results['E_pv']/data.loc[c, 'P_pv']/8760
    data.loc[c, 'capacity_factor_dispatch'] = results['E_dispatch']/data.loc[c, 'P_dispatch']/8760
    data.loc[c, 'share_dispatchable'] = results['E_dispatch']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data.loc[c, 'share_wind']=results['E_wind']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data.loc[c, 'share_pv']=results['E_pv']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    # data.loc[c, 'carbon_impact']=results['E_wind']*11+results['E_pv']*45/pop/1000

In [4]:
data_us = pd.read_csv('results/optimization_results_world_grid_with_states.csv')
data_us = data_us.set_index('State')

for col in data_us.columns[2:]: 
    data_us[f'{col}_normalized']=data_us[col]/data_us['mean_load']
# country_name = 'United States'
for i in range(len(data_us.index.to_list())): 
    c = data_us.index[i]
    country_name = data_us.iloc[i,0]
    results = pd.read_pickle(f'./results/{country_name}/optimization_results_{c}.pickle')
    iso_code = country_codes.loc[country_name,'Alpha-3 code' ].split(' ')[1]
    # pop = pop_data.loc[iso_code]
    # pop=pop.iloc[0]
    data_us.loc[c, 'capacity_factor_wind'] = results['E_wind']/data_us.loc[c, 'P_wind']/8760
    data_us.loc[c, 'capacity_factor_pv'] = results['E_pv']/data_us.loc[c, 'P_pv']/8760
    data_us.loc[c, 'capacity_factor_dispatch'] = results['E_dispatch']/data_us.loc[c, 'P_dispatch']/8760
    data_us.loc[c, 'share_dispatchable'] = results['E_dispatch']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data_us.loc[c, 'share_wind']=results['E_wind']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data_us.loc[c, 'share_pv']=results['E_pv']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    # data_us.loc[c, 'carbon_impact']=results['E_wind']*11+results['E_pv']*45/pop/1000

In [6]:
# shapefile_path = './shapefiles/geoBoundariesCGAZ_ADM1.shp'
# gdf = gpd.read_file(shapefile_path)

# gdf = gdf.to_crs(epsg=4326)

# gdf['geometry'] = gdf['geometry'].simplify(tolerance=0.01)

# fig, ax = plt.subplots(figsize=(10, 10))
# gdf.plot(ax=ax, linewidth=0.5, edgecolor='k', color='lightblue')

# ax.set_title('Your Title Here')

# ax.set_axis_off()

# plt.show()

In [5]:
fig = go.Figure(go.Choropleth(
    locations=data['iso_alpha'],
    z=data['P_dispatch_normalized'],
    text=data['iso_alpha'],
    hoverinfo='location+z+text',
    colorscale='Portland'
))

fig.update_layout(
    title_text='Dispatchable Capacity (relative to average load)',
    geo=dict(
        projection=go.layout.geo.Projection(type='natural earth')
    )
)
# Save the figure as an HTML file without displaying it
fig.write_html('choropleth_map_dispatch_capacity.html')

In [6]:
world = gpd.read_file('shapefiles/ne_110m_admin_0_countries.shp')

usa_gdf = gpd.read_file('shapefiles/USA/States_shapefile.shp')
usa_gdf = usa_gdf.merge(data_us, left_on='State_Code', right_on='State')  

canada_gdf = gpd.read_file('shapefiles/Canada/Canada_Provincial_boundaries_generalized.shp')
canada_gdf = canada_gdf.to_crs(world.crs)
canada_gdf = canada_gdf.merge(data_us, left_on='postal', right_on='State')  

In [7]:
china_gdf = gpd.read_file('shapefiles/China/chn_admbnda_adm1_ocha_2020.shp')
china_gdf = china_gdf.merge(data_us, left_on='ADM1_PCODE', right_on='State')
brazil_gdf = gpd.read_file('shapefiles/Brazil/geoBoundaries-BRA-ADM1.shp')
for i in brazil_gdf.index :
    brazil_gdf.loc[i,'shapeISO'] = brazil_gdf.loc[i,'shapeISO'].split('-')[1]
brazil_gdf = brazil_gdf.merge(data_us, left_on='shapeISO', right_on='State')

# india_gdf = gpd.read_file('shapefiles/China/chn_admbnda_adm1_ocha_2020.shp')
# india_gdf = india_gdf.merge(data_us, left_on='State_Code', right_on='State')
australia_gdf = gpd.read_file('shapefiles/Australia/georef-australia-state-millesime.shp')
australia_gdf = australia_gdf.merge(data_us, left_on='ste_iso3166', right_on='State')

russia_gdf = gpd.read_file('shapefiles/Russia/geoBoundaries-RUS-ADM1.shp')
russia_gdf["geometry"]=(russia_gdf.to_crs(russia_gdf.estimate_utm_crs()).simplify(1000).to_crs(russia_gdf.crs))
for i in russia_gdf.index :
    russia_gdf.loc[i,'shapeISO'] = russia_gdf.loc[i,'shapeISO'].split('-')[1]
russia_gdf = russia_gdf.to_crs(world.crs)
russia_gdf = russia_gdf.merge(data_us, left_on='shapeISO', right_on='State') 

In [8]:
fig = px.choropleth(data, locations='iso_alpha', color='P_dispatch_normalized', hover_name='iso_alpha',
                    projection='natural earth', title='Dispatchable capacity (relative to average load)', color_continuous_scale='portland',
                    hover_data={
                    'iso_alpha': True
         })
# Couche US states
fig.add_trace(px.choropleth(
    usa_gdf,
    geojson=usa_gdf.geometry,
    locations=usa_gdf.index,
    color='P_dispatch_normalized',
    hover_name='State_Name',
    color_continuous_scale='portland',
).data[0])
# Couche Canada provinces
fig.add_trace(px.choropleth(
    canada_gdf,
    geojson=canada_gdf.geometry.__geo_interface__,
    locations=canada_gdf.index,
    color='P_dispatch_normalized',
    hover_name='Name_EN',
    color_continuous_scale='portland',
).data[0])

fig.add_trace(px.choropleth(
    brazil_gdf,
    geojson=brazil_gdf.geometry.__geo_interface__,
    locations=brazil_gdf.index,
    color='P_dispatch_normalized',
    hover_name='shapeISO',
    color_continuous_scale='portland',
).data[0])

fig.add_trace(px.choropleth(
    russia_gdf,
    geojson=russia_gdf.geometry.__geo_interface__,
    locations=russia_gdf.index,
    color='P_dispatch_normalized',
    hover_name='shapeISO',
    color_continuous_scale='portland',
).data[0])
fig.add_trace(px.choropleth(
    australia_gdf,
    geojson=australia_gdf.geometry.__geo_interface__,
    locations=australia_gdf.index,
    color='P_dispatch_normalized',
    hover_name='ste_code',
    color_continuous_scale='portland',
).data[0])
fig.add_trace(px.choropleth(
    china_gdf,
    geojson=china_gdf.geometry.__geo_interface__,
    locations=china_gdf.index,
    color='P_dispatch_normalized',
    hover_name='ADM1_PCODE',
    color_continuous_scale='portland',
).data[0])
# Mettre à jour les géométries
fig.update_geos(fitbounds="locations", visible=False)
fig.write_html('choropleth_map_dispatch_capacity.html')

In [14]:
fig = px.choropleth(data, locations='iso_alpha', color='P_wind_normalized', hover_name='iso_alpha',
                    projection='natural earth', title='Wind capacity (relative to average load)', color_continuous_scale='portland',
                    range_color = [0,5],
                    hover_data={
                    'iso_alpha': True, 
                    'share_pv': True,    
                    'share_wind': True, 
                    'share_dispatchable': True 
         })
# Couche US states
fig.add_trace(px.choropleth(
    usa_gdf,
    geojson=usa_gdf.geometry,
    locations=usa_gdf.index,
    color='P_wind_normalized',
    hover_name='State_Name',
    color_continuous_scale='portland',
).data[0])
# Couche Canada provinces
fig.add_trace(px.choropleth(
    canada_gdf,
    geojson=canada_gdf.geometry.__geo_interface__,
    locations=canada_gdf.index,
    color='P_wind_normalized',
    hover_name='Name_EN',
    color_continuous_scale='portland',
).data[0])

fig.add_trace(px.choropleth(
    brazil_gdf,
    geojson=brazil_gdf.geometry.__geo_interface__,
    locations=brazil_gdf.index,
    color='P_wind_normalized',
    hover_name='shapeISO',
    color_continuous_scale='portland',
).data[0])

fig.add_trace(px.choropleth(
    russia_gdf,
    geojson=russia_gdf.geometry.__geo_interface__,
    locations=russia_gdf.index,
    color='P_wind_normalized',
    hover_name='shapeISO',
    color_continuous_scale='portland',
).data[0])
fig.add_trace(px.choropleth(
    australia_gdf,
    geojson=australia_gdf.geometry.__geo_interface__,
    locations=australia_gdf.index,
    color='P_wind_normalized',
    hover_name='ste_code',
    color_continuous_scale='portland',
).data[0])
fig.add_trace(px.choropleth(
    china_gdf,
    geojson=china_gdf.geometry.__geo_interface__,
    locations=china_gdf.index,
    color='P_wind_normalized',
    hover_name='ADM1_PCODE',
    color_continuous_scale='portland',
).data[0])
# Mettre à jour les géométries
fig.update_geos(fitbounds="locations", visible=False)
fig.write_html('choropleth_map_wind_capacity_installed.html')

In [15]:
fig = px.choropleth(data, locations='iso_alpha', color='P_pv_normalized', hover_name='iso_alpha',
                    projection='natural earth', title='PV Capacity (relative to average load)', color_continuous_scale='portland',
                    range_color = [0,5],
                    hover_data={
                    'iso_alpha': True, 
                    'share_pv': True,    
                    'share_wind': True, 
                    'share_dispatchable': True 
         })
# Couche US states
fig.add_trace(px.choropleth(
    usa_gdf,
    geojson=usa_gdf.geometry,
    locations=usa_gdf.index,
    color='P_pv_normalized',
    hover_name='State_Name',
    color_continuous_scale='portland',
).data[0])
# Couche Canada provinces
fig.add_trace(px.choropleth(
    canada_gdf,
    geojson=canada_gdf.geometry.__geo_interface__,
    locations=canada_gdf.index,
    color='P_pv_normalized',
    hover_name='Name_EN',
    color_continuous_scale='portland',
).data[0])

fig.add_trace(px.choropleth(
    brazil_gdf,
    geojson=brazil_gdf.geometry.__geo_interface__,
    locations=brazil_gdf.index,
    color='P_pv_normalized',
    hover_name='shapeISO',
    color_continuous_scale='portland',
).data[0])

fig.add_trace(px.choropleth(
    russia_gdf,
    geojson=russia_gdf.geometry.__geo_interface__,
    locations=russia_gdf.index,
    color='P_pv_normalized',
    hover_name='shapeISO',
    color_continuous_scale='portland',
).data[0])
fig.add_trace(px.choropleth(
    australia_gdf,
    geojson=australia_gdf.geometry.__geo_interface__,
    locations=australia_gdf.index,
    color='P_pv_normalized',
    hover_name='ste_code',
    color_continuous_scale='portland',
).data[0])
fig.add_trace(px.choropleth(
    china_gdf,
    geojson=china_gdf.geometry.__geo_interface__,
    locations=china_gdf.index,
    color='P_pv_normalized',
    hover_name='ADM1_PCODE',
    color_continuous_scale='portland',
).data[0])
# Mettre à jour les géométries
fig.update_geos(fitbounds="locations", visible=False)
fig.write_html('choropleth_map_pv_capacity_installed.html')